In [1]:
# Import libraries
import pandas as pd

# Reshaping data with .melt()

## <font color=blue>DATA</font>

In [2]:
inflation_wide = pd.read_excel('datasets/revised-seasonally-adjusted-indexes-2018.xlsx', 
                               engine='openpyxl', sheet_name=0, header=0)

data_type, item, seriesid = 'SEASONALLY ADJUSTED INDEX', 'SA0', 'CUSR0000SA0'
df_filter      = (inflation_wide.DATA_TYPE==data_type) & (inflation_wide.ITEM==item)  & (inflation_wide.seriesid==seriesid)
inflation_wide = inflation_wide[df_filter]

inflation_wide.head()

C:\Anaconda3\envs\datascience\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


,ITEM,TITLE,seriesid,DATA_TYPE,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC
0,SA0,All items,CUSR0000SA0,SEASONALLY ADJUSTED INDEX,2014,235.288,235.547,236.028,236.468,236.918,237.231,237.498,237.460,237.477,237.430,236.983,236.252
1,SA0,All items,CUSR0000SA0,SEASONALLY ADJUSTED INDEX,2015,234.718,235.236,236.005,236.156,236.974,237.684,238.053,238.028,237.506,237.781,238.016,237.817
2,SA0,All items,CUSR0000SA0,SEASONALLY ADJUSTED INDEX,2016,237.833,237.469,238.038,238.827,239.464,240.167,240.150,240.602,241.051,241.691,242.029,242.772
3,SA0,All items,CUSR0000SA0,SEASONALLY ADJUSTED INDEX,2017,243.780,243.961,243.749,244.051,243.962,244.182,244.390,245.297,246.418,246.587,247.332,247.901
4,SA0,All items,CUSR0000SA0,SEASONALLY ADJUSTED INDEX,2018,248.884,249.369,249.498,249.956,250.646,251.134,251.597,251.879,252.010,252.794,252.760,252.723


## <font color=blue>USING .stack()</font>

In [3]:
inflation_long = inflation_wide.set_index('YEAR')
df_date        = inflation_long.iloc[:,4:].stack()
inflation_long = pd.DataFrame({'seriesid' : seriesid,
                               'data_type': data_type,
                               'date'     : pd.to_datetime([str(y) + '-' + m + '-01' for y, m in df_date.index.values]),
                               'cpi'      : df_date.values
                              })
inflation_long.head()

,seriesid,data_type,date,cpi
0,CUSR0000SA0,SEASONALLY ADJUSTED INDEX,2014-01-01,235.288
1,CUSR0000SA0,SEASONALLY ADJUSTED INDEX,2014-02-01,235.547
2,CUSR0000SA0,SEASONALLY ADJUSTED INDEX,2014-03-01,236.028
3,CUSR0000SA0,SEASONALLY ADJUSTED INDEX,2014-04-01,236.468
4,CUSR0000SA0,SEASONALLY ADJUSTED INDEX,2014-05-01,236.918


## <font color=blue>USING .melt()</font>

In [4]:
inflation_long = inflation_wide.melt(id_vars=['seriesid','DATA_TYPE', 'YEAR'], 
                                     value_vars=['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 
                                                 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC'],
                                     var_name=['month'], value_name='cpi')
#inflation_long['date'] = pd.to_datetime([str(y) + '-' + m + '-01' for y, m in inflation_long[['YEAR', 'month']].values])
inflation_long['date'] = pd.to_datetime(inflation_long.YEAR.astype(str) + '-' + inflation_long.month)
inflation_long = inflation_long[['seriesid', 'DATA_TYPE', 'date', 'cpi']]
inflation_long.columns = ['seriesid', 'data_type', 'date', 'cpi']
inflation_long.head()

,seriesid,data_type,date,cpi
0,CUSR0000SA0,SEASONALLY ADJUSTED INDEX,2014-01-01,235.288
1,CUSR0000SA0,SEASONALLY ADJUSTED INDEX,2015-01-01,234.718
2,CUSR0000SA0,SEASONALLY ADJUSTED INDEX,2016-01-01,237.833
3,CUSR0000SA0,SEASONALLY ADJUSTED INDEX,2017-01-01,243.780
4,CUSR0000SA0,SEASONALLY ADJUSTED INDEX,2018-01-01,248.884
